# GooglePalm Chatbox for Online Shop

This is an end to end LLM project based on Google Palm and Langchain. We are building a system that can interact with MySQL database and answer the related questions. 
User asks questions in a natural language and the system generates answers by converting those questions to an SQL query and
then executing that query on MySQL database. 
Rude Manners is a T-shirt store where they maintain their inventory, sales and discounts data in MySQL database. A store manager 
will may ask questions such as,
- How many white color Adidas t shirts do we have left in the stock?
- How much sales our store will generate if we can sell all extra-small size t shirts after applying discounts?
The system is intelligent enough to generate accurate queries for given question and execute them on MySQL database

In [6]:
#Importing libraries
import json
from langchain.llms import GooglePalm

#Setting up googlepalm and mysql API
with open("keys.json") as f:
    keys = json.load(f)

api_key = keys["api_key"]
llm = GooglePalm(google_api_key=api_key, temperature=0.2)




In [7]:
#Checking the connection
advice = llm("whats meaning of life in 1 sentense words")
print(advice)

/Users/sultanbeishenkulov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


To live and to love.


In [8]:
from langchain.utilities import SQLDatabase

# Extract the database credentials
db_user = keys['db_user']
db_password = keys['db_password']
db_host = keys['db_host']
db_name = keys['db_name']

#Connecting to local MySQL database in which we have our shop data
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [9]:
from langchain_experimental.sql import SQLDatabaseChain

#Connecting our database to googlepalm through Langchain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

#Cheking the accuracy of the LLM
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



/Users/sultanbeishenkulov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(40,)]
Answer:40
> Finished chain.


In [10]:
qns1


{'query': 'How many t-shirts do we have left for nike in extra small size and white color?',
 'result': '40'}

In [11]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")

/Users/sultanbeishenkulov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('386'),)]
Answer:386
> Finished chain.


> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('20972'),)]
Answer:20972
> Finished chain.


In [12]:
# qns3 = db_chain.run(
#     "SELECT ROUND(SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)), 2) AS total_revenue " +
#     "FROM (SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id " +
#     "FROM t_shirts " +
#     "WHERE brand = 'Levi' " +
#     "GROUP BY t_shirt_id) a " +
#     "LEFT JOIN discounts " +
#     "ON a.t_shirt_id = discounts.t_shirt_id"
# )
# print(qns3)
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")

#We see that this one does not work, our LLM does not understand what to do and uses start_date and end_date which we don't have in our database which occurs the error



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23643.000000'),)]
Answer:23643
> Finished chain.


In [ ]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('23811'),)]
Answer:23811
> Finished chain.


In [ ]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(27,), (67,), (82,), (25,)]
Answer:27
> Finished chain.


In [ ]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('201'),)]
Answer:201
> Finished chain.


### Few Shot Learning

In [13]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from few_shots import few_shots

#We need to embedd our promts in order for LLM to learn 
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#Converting our list of dictionaries to list of strings for the embeddings 
to_vectorize = [
    " ".join(
        str(value) if not isinstance(value, dict) else " ".join(f"{k}: {v}" for k, v in value.items())
        for value in example.values()
    )
    for example in few_shots
]
to_vectorize


/Users/sultanbeishenkulov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 40",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 20972",
 "If we have to sell all the Levis T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 23643",
 "If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Resu

In [14]:
from langchain.vectorstores import Chroma

#Created and stored our embeddings in ChromaDB
vectorstore = Chroma.from_texts(
    to_vectorize, 
    embedding=embeddings, 
    metadatas=few_shots
)
print(vectorstore)

In [15]:
#Looking for similar promts in our embedding space

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '40',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '23811',
  'Question': 'If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [16]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [17]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [18]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [19]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [20]:
#Creating prompt template for new LLM connection to actually learn from our few shot file
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [21]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [22]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('201'),)]
Answer:201
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '201'}

In [23]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('20972'),)]
Answer:20972
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '20972'}

In [24]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('7475'),)]
Answer:7475
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '7475'}

In [25]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25662.900000'),)]
Answer:25662.9
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '25662.9'}

In [26]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23365.550000'),)]
Answer:23365.55
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '23365.55'}

In [27]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('23588'),)]
Answer:23588
> Finished chain.


'23588'

#### As a result we see that after adding prompt template our chatbox does not make the same mistakes it did in the beggining. It is a good idea to check other possible questions and check the use cases and add new prompts to few shot for our LLM to give right answers.